# Conditional Kaplan Meier

## Stage 2: Dependent Censoring

Before running the notebook make sure that:

1. Created a python/conda environment according to the specifications
2. Set your working directory (`project_dir`) to the directory where you unpacked the .zip file
3. You might want to add a shebang line to the ./utils (however using it from notebooks with a specified environment should work as well)

The notebook genereates data, trains models and saves them into the ./data directory. Note that at times you will need to overwrite existing files (this is to ensure that you do not accidentally overwrite files that took a long time training - aka. model weights)

See R-Scrips for visualisations (by default, all visualisations should already be in the zip folder)

In [1]:
project_dir = ''

In [2]:
# Housekeeping
import os
os.chdir(project_dir)

# For data generation
import numpy as np
import pandas as pd
from scipy.stats import gamma, bernoulli, multinomial
from scipy.integrate import simps
from scipy.special import expit
from itertools import product as itp

# For standard survival modelling
from lifelines import KaplanMeierFitter, CoxPHFitter
import statsmodels.api as sm

# Model loading
import tensorflow as tf

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

tf.keras.utils.set_random_seed(42)

In [3]:
%load_ext autoreload
%autoreload 2

from main.utils.conditional_km import DeepKaplanMeier, SimpleDeepKaplanCensoring
from main.utils.metrics import calculate_concordance_surv
train_ = False

## Running conditional and nonlinear conditional model

In [4]:
size_ = 3000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = -3.14*x_1 + 0.318*x_2 + 2.72*x_3

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(2*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [5]:
kaplan_meier = KaplanMeierFitter()
kaplan_meier.fit(variable_dict['observed_survival'], 
                 ~variable_dict['censoring'])

kaplan_meier_survival = kaplan_meier.survival_function_

# Export for plotting
(kaplan_meier_survival.reset_index()
                      .to_csv('data/dependent_censoring/kaplan_meier_linearly_dep_censoring_1.csv', 
                      index=False))

In [6]:
# Set up model
total_periods = int(variable_dict['observed_survival'].max())

deep_model_linear_dep = DeepKaplanMeier(total_periods)
deep_model_linear_dep.compile_model((3, ), [12,12])

2022-10-20 17:50:03.704010: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-20 17:50:03.704114: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:
# Fit model
if train_:
    deep_model_linear_dep.fit_model(variable_dict['features'],
                                variable_dict['observed_survival'].astype(int), 
                                variable_dict['censoring'],
                                epochs=100, 
                                verbose=False, 
                                batch_size=128
                                )
    mod_ = deep_model_linear_dep.model

else:
    mod_ = tf.keras.models.load_model('data/models/keras_model_simple_lin_cens_.keras')

OSError: No file or directory found at data/models/keras_model_simple_lin_cens_.keras

In [ ]:
preds_ = mod_.predict(variable_dict['features'])
preds_ = pd.DataFrame(list(map(np.ravel, preds_)))

# Predictions are a list for each timeperiod, flatten and create exportable dataframe
ckm_df = (preds_
          .reset_index()
          .rename(columns={'index':'timeline'}))
# Export
ckm_df.to_csv('data/dependent_censoring/ckm_linear_dependent_censoring.csv', 
              index=False)

nom_, denom_, val_ = calculate_concordance_surv(variable_dict['observed_survival'], 
                                                variable_dict['censoring'], 
                                                preds_)
print(f"The concordance index is: {np.round(val_, 3)}")

2022-10-20 17:12:48.678042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 2s 18ms/step
The concordance index is: 0.806


In [ ]:
true_data = (pd.DataFrame({'surv_time' : variable_dict['true_survival']})
             .groupby('surv_time')
             .agg({'surv_time': 'count'})
             .rename(columns={'surv_time': 'failures'})
             .reset_index())

true_data['relative'] = true_data.failures / size_
true_data['survival'] = 1-np.cumsum(true_data.relative)
true_data = true_data[['surv_time', 'survival']]
true_data = (true_data
             .append(pd.DataFrame({'surv_time': [0], 'survival':[1]}))
             .reset_index()
             .sort_values('surv_time'))

(true_data[['surv_time', 'survival']].rename(columns={'surv_time': 'timeline'})
                                     .to_csv('data/dependent_censoring/true_data_linear_censoring.csv', 
                                     index=False))

/var/folders/jy/l1h7n_nx6sl3h96lm_0n3ppc0000gn/T/ipykernel_6846/1256711721.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  true_data = (true_data


In [ ]:
# Cox model
cox_df = (pd.DataFrame(
                       np.column_stack((variable_dict['observed_survival'],
                                        ~variable_dict['censoring'],
                                        variable_dict['features']))
                       ))
cox_df.columns = ['surv', 'cens', 'feat_1', 'feat_2', 'feat_3']

cph = CoxPHFitter()
cph.fit(cox_df, duration_col='surv', event_col='cens')


<lifelines.CoxPHFitter: fitted with 3000 total observations, 800 right-censored observations>

In [ ]:
print(f"Cox concordance index for Linear model is: {np.round(cph.concordance_index_,3)}")

Cox concordance index for Linear model is: 0.805


### Nonlinear Hazards

In [ ]:
size_ = 3000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = np.sin(x_1)*3.14 + 0.318*x_2 + 2.72*np.abs(np.cos(x_3))

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(2*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [ ]:
# Create some test data as well
size_ = 1000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = np.sin(x_1)*3.14 + 0.318*x_2 + 2.72*np.abs(np.cos(x_3))

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(2*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict_test = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [ ]:
# Cox model
cox_df = (pd.DataFrame(
                       np.column_stack((variable_dict['observed_survival'],
                                        ~variable_dict['censoring'],
                                        variable_dict['features']))
                       ))
cox_df.columns = ['surv', 'cens', 'feat_1', 'feat_2', 'feat_3']

cox_df_test = (pd.DataFrame(
                       np.column_stack((variable_dict_test['observed_survival'],
                                        ~variable_dict_test['censoring'],
                                        variable_dict_test['features']))
                       ))
cox_df_test.columns = ['surv', 'cens', 'feat_1', 'feat_2', 'feat_3']

cph = CoxPHFitter()
cph.fit(cox_df, duration_col='surv', event_col='cens')


<lifelines.CoxPHFitter: fitted with 3000 total observations, 104 right-censored observations>

In [ ]:
print(f"Cox concordance index is: {np.round(cph.concordance_index_,2)}")

Cox concordance index is: 0.59


In [ ]:
if train_:
    # Make model quite flexible 
    tf.keras.utils.set_random_seed(42)

    # Set up model
    total_periods = int(variable_dict['observed_survival'].max())

    deep_model_simple = DeepKaplanMeier(total_periods)
    deep_model_simple.compile_model((3, ), [20,20,20], preoutput=12)

    # Fit model
    deep_model_simple.fit_model(variable_dict['features'],
                                variable_dict['observed_survival'].astype(int), 
                                variable_dict['censoring'],
                                epochs=150, 
                                verbose=False, 
                                batch_size=128
                                )
    mod_s = deep_model_linear_dep.model
else:
    mod_s = tf.keras.models.load_model('data/model_weights/model_nonlinear_dep_cens.keras')

In [ ]:
preds_test = mod_s.predict(variable_dict_test['features'])
preds_test = pd.DataFrame(list(map(np.ravel, preds_test)))

nom_, denom_, val_ = calculate_concordance_surv(variable_dict_test['observed_survival'], 
                                                variable_dict_test['censoring'], 
                                                preds_test)
print(val_)

2022-10-20 17:08:39.177835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 12ms/step
0.6436078213290981


## Robustness trials with size

Here we increase the censoring proportion (by decreasing the scale parameter to 0.75) - see line 17 in the cell below

In [ ]:
size_ = 3000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = -3.14*x_1 + 0.318*x_2 + 2.72*x_3

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(0.75*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [ ]:
print(f"Censoring proportion is {np.round(variable_dict['censoring'].mean(), 3)}")

Censoring proportion is 0.713


In [ ]:
if train_:
        
    tf.keras.utils.set_random_seed(42)

    # Set up model
    total_periods = int(variable_dict['observed_survival'].max())

    model_heavy_cens = DeepKaplanMeier(total_periods)
    model_heavy_cens.compile_model((3, ), [12,12], preoutput=12)

    # Fit model
    model_heavy_cens.fit_model(variable_dict['features'],
                                variable_dict['observed_survival'].astype(int), 
                                variable_dict['censoring'],
                                epochs=50, 
                                verbose=False, 
                                batch_size=128
                                )

    mod_hcs = model_heavy_cens.model

else:
    mod_hcs = tf.keras.models.load_model('data/model_weights/model_heavy_censoring_small.keras')

In [ ]:
# Run predictions 
predictions_ckm_hc = mod_hcs.predict(variable_dict['features'])

# Predictions are a list for each timeperiod, flatten and create exportable dataframe
ckm_df = (pd.DataFrame(list(map(np.ravel, predictions_ckm_hc)))
          .reset_index()
          .rename(columns={'index':'timeline'}))

# Avoid Python specific index issues
ckm_df['timeline'] = ckm_df.timeline

# Export
ckm_df.to_csv('data/dependent_censoring/ckm_pred_heavy_censoring_small.csv', 
              index=False)

2022-10-20 17:14:09.730747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 3s 21ms/step


In [ ]:
true_data = (pd.DataFrame({'surv_time' : variable_dict['true_survival']})
             .groupby('surv_time')
             .agg({'surv_time': 'count'})
             .rename(columns={'surv_time': 'failures'})
             .reset_index())

true_data['relative'] = true_data.failures / size_
true_data['survival'] = 1-np.cumsum(true_data.relative)
true_data = true_data[['surv_time', 'survival']]
true_data = (true_data
             .append(pd.DataFrame({'surv_time': [0], 'survival':[1]}))
             .reset_index()
             .sort_values('surv_time'))

(true_data[['surv_time', 'survival']].rename(columns={'surv_time': 'timeline'})
                                     .to_csv('data/dependent_censoring/heavy_censoring_true.csv', 
                                     index=False))

/var/folders/jy/l1h7n_nx6sl3h96lm_0n3ppc0000gn/T/ipykernel_6846/3134575718.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  true_data = (true_data


In [ ]:
kaplan_meier = KaplanMeierFitter()
kaplan_meier.fit(variable_dict['observed_survival'], 
                 ~variable_dict['censoring'])

kaplan_meier_survival = kaplan_meier.survival_function_

# Export for plotting
(kaplan_meier_survival.reset_index()
                      .to_csv('data/dependent_censoring/km_heavy_censoring.csv', 
                      index=False))

## Increase Dataset size

In [ ]:
size_ = 12000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = -3.14*x_1 + 0.318*x_2 + 2.72*x_3

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(0.75*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [ ]:
if train_:
    # Set up model
    total_periods = int(variable_dict['observed_survival'].max())

    model_heavy_cens = DeepKaplanMeier(total_periods)
    model_heavy_cens.compile_model((3, ), [24,24,24], preoutput=24)

    # Fit model
    model_heavy_cens.fit_model(variable_dict['features'],
                            variable_dict['observed_survival'].astype(int), 
                            variable_dict['censoring'],
                            epochs=50, 
                            verbose=False, 
                            batch_size=128
                            )
    mod_hcl = model_heavy_cens.model
else:
    mod_hcl = tf.keras.models.load_model('data/model_weights/model_heavy_censoring_large.keras')

In [ ]:
# Run predictions 
predictions_ckm_hc = mod_hcl.predict(variable_dict['features'])

# Predictions are a list for each timeperiod, flatten and create exportable dataframe
ckm_df = (pd.DataFrame(list(map(np.ravel, predictions_ckm_hc)))
          .reset_index()
          .rename(columns={'index':'timeline'}))

# Avoid Python specific index issues
ckm_df['timeline'] = ckm_df.timeline

# Export
ckm_df.to_csv('data/dependent_censoring/ckm_pred_heavy_censoring_larger_model.csv', 
              index=False)

2022-10-20 17:15:10.664209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 6s 14ms/step


## "Manual" approach

In [8]:
size_ = 3000
seed_ = 42

np.random.seed(seed_)
x_1 = np.random.exponential(0.1, size=size_)
x_2 = np.random.normal(10, np.sqrt(5), size=size_)
x_3 = np.random.poisson(5, size=size_)

features = np.array([x_1, x_2, x_3]).T

# Arrange Targets
survial_model = -3.14*x_1 + 0.318*x_2 + 2.72*x_3

true_survival_time = gamma.rvs(survial_model, scale=1, random_state=seed_)
ceiling_surv_ = np.ceil(true_survival_time)

censoring_transformed = np.minimum(0.75*(survial_model/survial_model.max()),1)

censored_instances = np.random.uniform(size=size_) > (censoring_transformed)
censored_surv = [np.ceil(np.random.uniform()*val_) for val_ in ceiling_surv_[censored_instances]]

observed_survival = ceiling_surv_.copy()
observed_survival[censored_instances] = censored_surv

variable_dict = dict({'true_survival': ceiling_surv_,
                      'observed_survival': observed_survival, 
                      'censoring': censored_instances,
                      'features' :  features})

In [9]:
if train_:
        
    # Set up model
    total_periods = int(variable_dict['observed_survival'].max())

    model_censoring = DeepKaplanMeier(total_periods)
    model_censoring.compile_model((3, ), [12,12])

    censoring_censroing = np.repeat(False, len(variable_dict['censoring']))


    _, inverse_censoring_matrix = model_censoring.prepare_survival(variable_dict['observed_survival'].astype(int),
                                                                variable_dict['censoring'])
    censoring_matrix = 1-inverse_censoring_matrix

    censoring_times = total_periods - censoring_matrix.sum(axis=1) + 1

    # Fit model
    model_censoring.fit_model(variable_dict['features'],
                            censoring_times.astype(int), 
                            censoring_censroing,
                            epochs=50, 
                            verbose=False, 
                            batch_size=64)

    preds_censoring = model_censoring.model.predict(variable_dict['features'])
    weight_matrix_censoring = np.hstack(preds_censoring).mean(axis=0) / np.hstack(preds_censoring)

    total_periods = int(variable_dict['observed_survival'].max())

    model_survival = SimpleDeepKaplanCensoring(total_periods)
    model_survival.compile_model((3, ), [20,20])

    model_survival.fit_model(variable_dict['features'],
                                variable_dict['observed_survival'].astype(int), 
                                variable_dict['censoring'],
                                weight_matrix_censoring,
                                epochs=50, 
                                verbose=False, 
                                batch_size=256
                                )

    mod_manual = model_survival.model

else:
    mod_manual = tf.keras.models.load_model('data/model_weights/model_manual_survival.keras')    

In [11]:
# Run predictions 
predictions_ckm = mod_manual.predict(variable_dict['features'])

# Predictions are a list for each timeperiod, flatten and create exportable dataframe
ckm_df = (pd.DataFrame(list(map(np.ravel, predictions_ckm)))
          .reset_index()
          .rename(columns={'index':'timeline'}))

# Avoid Python specific index issues
ckm_df['timeline'] = ckm_df.timeline

# Export
ckm_df.to_csv('data/dependent_censoring/manually_corrected_version_dependent.csv', 
              index=False)

94/94 [==============================] - 2s 18ms/step
